In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
from scipy.integrate import solve_ivp

## Heavy, symmetric top

An important problem in rigid body dynamics is the heavy symmetrical top with one point fixed which we investigated in lecture.  For this problem $I_1=I_2\ne I_3$.

Recall the top is uniform and has one point on the symmetry ($z$-) axis fixed, so it's configuration is completely given by the Euler angles: $\theta$ gives the inclination of the top to the vertical, $\phi$ the azimuth of the top about the vertical and $\psi$ the rotation angle about the top axis.  As the top moves its motion in $\theta$ is called "nutation" while its motion in $\phi$ is "precession".

Then from class we have
$$
  T = \frac{1}{2} I_1 \left(\dot{\theta}^2+\sin^2\theta\dot{\phi}^2\right)
    + \frac{1}{2} I_3 \left(\dot{\psi}+\cos\theta\dot{\phi}\right)^2
$$
while the potential energy is just that due to the center of mass
$$
  V = Mg\ell\cos\theta
$$
if the center of mass is $\ell$ from the fixed point.

The Lagrangian is independent of $\phi$ and $\psi$ so we have two conserved (conjugate) momenta:
$$
  p_\theta = \left( I_1\sin^2\theta + I_3\cos^2\theta\right)\,\dot{\phi} + I_3\cos\theta\ \dot{\psi}
  \quad\mathrm{and}\quad
  p_\psi = I_3\left( \dot{\psi} + \dot{\phi}\cos\theta \right)
$$
The $p_\psi$ equation says that given $\theta(t)$ and $\phi(t)$ we can solve for $\psi(t)$.  Since we don't really care about $\psi$ for how the top moves we shall "neglect" this equation (knowing we could solve it if we ever needed $\psi$).

We can solve for $\dot{\phi}$ by noting that
$$
  p_\phi - p_\psi\,\cos\theta = I_1\sin^2\theta\,\dot{\phi} + I_3\cos^2\theta\,\dot{\phi} + I_3\cos\theta\,\dot{\psi}
  - I_3\cos\theta\,\dot{\psi} - I_3\cos^2\theta\,\dot{\phi}
  = I_1\sin^2\theta\,\dot{\phi}
$$
If we define $p_\psi \equiv I_1 a$ and $p_\phi \equiv I_1 b$ with $a$ and $b$ constants then
$$
  \dot{\phi}
  = \frac{p_\phi-p_\psi\,\cos\theta}{I_1\,\sin^2\theta}
  = \frac{b-a\cos\theta}{\sin^2\theta}
$$
Thus once we know $\theta(t)$ we can integrate this to get $\phi(t)$ -- note
depending upon the relative size of $a$ and $b$ we could have $\dot{\phi}$
be positive or negative or even change sign!
Energy conservation implied that $\theta$ moves in an effective 1D potential
$$
  V_{\rm eff}(\theta) = Mg\ell\cos\theta
  + \frac{I_1}{2}\left( \frac{b-a\cos\theta}{\sin\theta} \right)^2
$$
and we can always choose units where $I_1=1$.  Comparing the expression for $V_{\rm eff}$ above to the Hamiltonian we derived in class you should be able to see the mapping explicitly.

In [ ]:
# Define the effective potential:
def Veff(tt,a,b,Mgl):
    return( Mgl*np.cos(tt)+0.5*( (b-a*np.cos(tt))/np.sin(tt) )**2 )

In [ ]:
# Just plot the effective potential for some values of a, b and Mgl.
#
tt     = np.linspace(0.01,0.99*np.pi,500)
fig,ax = plt.subplots(1,1,figsize=(6,4))
for pars in [ [0.0,1.0,0.0],[0.5,1.0,0.0],[0.5,1.0,1.0],\
             [0.5,1.0,2.0], [1.0,0.65,0.1] ]:
    a,b,Mgl = pars
    lbl     = '$a={:.2f}$, '.format(a) +\
              '$b={:.2f}$, '.format(b) +\
              r'$Mg\ell='+'{:.2f}$'.format(Mgl)
    ax.plot(tt/np.pi,Veff(tt,a,b,Mgl),label=lbl)
#
ax.legend()
#
ax.set_yscale('log')
ax.set_xlabel(r'$\theta/\pi$')
ax.set_ylabel(r'$V_{\rm eff}(\theta)$')
plt.tight_layout()

In [ ]:
# Make exactly the same plot, but now make the x-axis cos(theta).
#
tt     = np.linspace(0.01,0.99*np.pi,500)
cost   = np.cos(tt)
fig,ax = plt.subplots(1,1,figsize=(6,4))
for pars in [ [0.0,1.0,0.0],[0.5,1.0,0.0],[0.5,1.0,1.0],\
             [0.5,1.0,2.0], [1.0,0.65,0.1] ]:
    a,b,Mgl = pars
    lbl     = '$a={:.2f}$, '.format(a) +\
              '$b={:.2f}$, '.format(b) +\
              r'$Mg\ell='+'{:.2f}$'.format(Mgl)
    ax.plot(cost,Veff(tt,a,b,Mgl),label=lbl)
#
ax.legend()
#
ax.set_yscale('log')
ax.set_xlabel(r'$\cos\theta$')
ax.set_ylabel(r'$V_{\rm eff}(\theta)$')
plt.tight_layout()

In [ ]:
# Pick a configuration and "energy":
a,b,Mgl = 1.0,0.65,0.1
#
theta0  = 0.2*np.pi
E0      = Veff(theta0,a,b,Mgl)
print("Energy E0={:f}".format(E0))

Now define the equations of motion, for which we need $-dV/d\theta$.  We will pack our y array as $(\theta,\dot{\theta},\phi)$.

In [ ]:
def derivs(t,y,a=0,b=0,Mgl=0):
    dy    = np.zeros_like(y)
    sint  = np.sin(y[0])
    cost  = np.cos(y[0])
    bac   = b-a*cost
    accn  = Mgl*sint - a*bac/sint
    accn += bac**2*cost/sint**3
    #
    dy[0] = y[1]
    dy[1] = accn
    dy[2] = bac/sint**2
    return(dy)

In [ ]:
times = np.linspace(0,100,500)
y0    = [theta0,0,0]     # Initial conditions.
res   = solve_ivp(derivs,[times[0],times[-1]],y0,\
                  method='DOP853',t_eval=times,args=(a,b,Mgl))
print(res)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
#
ax.plot(res.y[-1,:]/2/np.pi,np.cos(res.y[0,:]),color='C0')
#
ax.set_xlabel(r'$\phi/2\pi$')
ax.set_ylabel(r'$\cos\theta$')
fig.tight_layout()

Let's plot this on the surface of a sphere.

In [ ]:
# Draw a 3D figure showing the motion of the tip of the top.
# Note that we're neglecting the difference between z-x-z and
# z-y-z Euler conventions because they're just a rotation here.
fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(111, projection='3d')
# Draw the sphere on in light grey.
if True:
    N = 20
    theta, phi = np.linspace(0,np.pi,N),np.linspace(0,2*np.pi,2*N)
    theta, phi = np.meshgrid(theta, phi)
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    ax.plot_surface(x,y,z,alpha=.2,facecolors=[["w"]*N]*2*N)
# and then the tip of the top
x = np.sin(res.y[0,:])*np.cos(res.y[-1,:])
y = np.sin(res.y[0,:])*np.sin(res.y[-1,:])
z = np.cos(res.y[0,:])
ax.plot(x,y,z,lw=2,color='C1')
#
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-1.1,1.1])
ax.set_zlim([-1.1,1.1])
ax.set_aspect('equal')
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

Note that the axis of rotation of the top "nutates" between a minimum and maximum value of $\theta$ while it precesses in $\phi$.  In this case the precession can be in the positive or negative direction, allowing "loops".

Try another configuration, to explore another kind of behavior.

It is not uncommon to have the top spinning very fast ($a\gg 1$) and initially start with $\dot{\theta}=\dot{\phi}=0$ at some $\theta_0$ and $\phi_0$.  Without loss of generality we can set $\phi_0=0$.  If $I_1=1$, $a=5$, $Mg\ell=3$ and the top starts at $\cos\theta_0=0.75$, plot the path of the top i.e. plot $\cos\theta(t)$ vs. $\phi(t)/2\pi$.

The trajectory moves back and forth between $\theta_{\rm min}=\theta_0$ and $\theta_{\rm max}$.  What is the "shape" of the curve near $\theta=\theta_0$?

# The End